In [4]:
import torch
import torch.nn as nn

import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Data

In [10]:
df = pd.read_csv("planets_diff.csv", sep=";")
df.head()

,earth,sun,moon,venus,mars,jupiter
0,0.001482,0.001482,0.018272,0.00183,0.001085,0.000199
1,0.001482,0.001482,0.018316,0.00183,0.001085,0.000198
2,0.001482,0.001482,0.018361,0.00183,0.001085,0.000198
3,0.001482,0.001482,0.018407,0.00183,0.001085,0.000198
4,0.001482,0.001482,0.018454,0.00183,0.001085,0.000198


In [13]:
earth_data = df.earth.values.astype(float)
earth_data

array([0.00148241, 0.00148243, 0.00148244, ..., 0.00142615, 0.00142609,
       0.00142603])

In [14]:
len(earth_data)

220368

Split dataset

In [15]:
test_data_size = 20368

train_data = earth_data[:-test_data_size]
test_data = earth_data[-test_data_size:]

In [17]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(train_data .reshape(-1, 1))

Create tensor

In [19]:
train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1)

Create in out sequences

In [32]:
train_window = 20

def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [33]:
train_inout_seq = create_inout_sequences(train_data_normalized, train_window)

In [34]:
train_inout_seq[:3]

[(tensor([0.9849, 0.9851, 0.9854, 0.9856, 0.9859, 0.9861, 0.9864, 0.9866, 0.9868,
          0.9871, 0.9873, 0.9875, 0.9878, 0.9880, 0.9882, 0.9884, 0.9887, 0.9889,
          0.9891, 0.9893]), tensor([0.9895])),
 (tensor([0.9851, 0.9854, 0.9856, 0.9859, 0.9861, 0.9864, 0.9866, 0.9868, 0.9871,
          0.9873, 0.9875, 0.9878, 0.9880, 0.9882, 0.9884, 0.9887, 0.9889, 0.9891,
          0.9893, 0.9895]),
  tensor([0.9897])),
 (tensor([0.9854, 0.9856, 0.9859, 0.9861, 0.9864, 0.9866, 0.9868, 0.9871, 0.9873,
          0.9875, 0.9878, 0.9880, 0.9882, 0.9884, 0.9887, 0.9889, 0.9891, 0.9893,
          0.9895, 0.9897]), tensor([0.9899]))]

# Create LSTM Model

In [35]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = \
            self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [36]:
model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model

LSTM(
  (lstm): LSTM(1, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)

## Train THe Model

In [40]:
epochs = 150

for i in range(epochs):
    for seq, labels in train_inout_seq[:-1000]:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

KeyboardInterrupt: 